# Homework 2 - Generalized Hough Transform

## Theory

< Insert your answers here >

## Programming

Find object in an image using a template:  
![title](data/template.jpg)
![title](data/query.jpg)

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import cv2
import utils
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

def nonMaxSuprression(img, d=5):
    """
    Given an image set all values to 0 that are not
    the maximum in its (2d+1,2d+1)-window

    Parameters
    ----------
    img : ndarray
        an image
    d : int
        for each pixel consider the surrounding (2d+1,2d+1)-window

    Returns
    -------
    result : ndarray
    """
    rows, cols = img.shape
    result = np.zeros((rows, cols))
    for i in range(d, rows-d):
        for j in range(d, cols-d):
            local_max = np.max(img[i-d:i+d+1, j-d:j+d+1])
            if img[i, j] == local_max:
                result[i, j] = img[i, j]
    return result

def calcBinaryMask(img, thresh=0.3):
    """
    Compute the gradient of an image and compute a binary mask
    based on the threshold. Corresponds to O^B in the slides.

    Parameters
    ----------
    img : ndarray
        an image
    thresh : float
        A threshold value. The default is 0.3.

    Returns
    -------
    binary : ndarray
        A binary image.
    """
    # Compute gradients
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    # Threshold gradients
    binary = np.zeros_like(gradient_magnitude)
    binary[gradient_magnitude > thresh] = 1

    return binary

def correlation(img, template):
    """
    Compute a correlation of gradients between an image and a template.

    Note:
    You should use the formula in the slides using the fourier transform.
    Then you are guaranteed to succeed.

    However, you can also compute the correlation directly.
    The resulting image must have high positive values at positions
    with high correlation.

    Parameters
    ----------
    img : ndarray
        a grayscale image
    template : ndarray
        a grayscale image of the template

    Returns
    -------
    ndarray
        an image containing the correlation between image and template gradients.
    """
    # Compute gradient of the image
    grad_x_img = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y_img = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_img = np.sqrt(grad_x_img**2 + grad_y_img**2)

    # Compute gradient of the template
    grad_x_template = cv2.Sobel(template, cv2.CV_64F, 1, 0, ksize=3)
    grad_y_template = cv2.Sobel(template, cv2.CV_64F, 0, 1, ksize=3)
    gradient_template = np.sqrt(grad_x_template**2 + grad_y_template**2)

    # Copy template gradient into larger frame
    template_padded = np.zeros_like(gradient_img)
    template_padded[:gradient_template.shape[0], :gradient_template.shape[1]] = gradient_template

    # Apply a circular shift
    template_padded = np.roll(template_padded, -gradient_template.shape[0]//2, axis=0)
    template_padded = np.roll(template_padded, -gradient_template.shape[1]//2, axis=1)

    # Normalize template
    template_padded /= np.linalg.norm(template_padded)

    # Compute correlation using Fourier transform
    corr = cv2.filter2D(gradient_img, -1, template_padded)

    return corr

def GeneralizedHoughTransform(img, template, angles, scales):
    """
    Compute the generalized hough transform. Given an image and a template.

    Parameters
    ----------
    img : ndarray
        A query image
    template : ndarray
        a template image
    angles : list[float]
        A list of angles provided in degrees
    scales : list[float]
        A list of scaling factors

    Returns
    -------
    hough_table : list[(correlation, angle, scaling)]
        The resulting hough table is a list of tuples.
        Each tuple contains the correlation and the corresponding combination
        of angle and scaling factors of the template.

        Note the order of these values.
    """
    hough_table = []
    for angle in angles:
        for scale in scales:
            # Rotate and scale template
            rows, cols = template.shape
            M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, scale)
            rotated_scaled_template = cv2.warpAffine(template, M, (cols, rows))

            # Compute the correlation
            corr = correlation(img, rotated_scaled_template)

            # Store results with parameters in a list
            hough_table.append((corr, angle, scale))

    return hough_table


ModuleNotFoundError: No module named 'utils'

# Main Program

In [ ]:
# Load query image and template
query = cv2.imread("data/query.jpg", cv2.IMREAD_GRAYSCALE)
template = cv2.imread("data/template.jpg", cv2.IMREAD_GRAYSCALE)

# Visualize images
utils.show(query)
utils.show(template)

# Create search space and compute GHT
angles = np.linspace(0, 360, 36)
scales = np.linspace(0.9, 1.3, 10)
ght = GeneralizedHoughTransform(query, template, angles, scales)

# extract votes (correlation) and parameters
votes, thetas, s = zip(*ght)

# Visualize votes
print("Hough votes")
votes = np.stack(votes).max(0)
plt.imshow(votes)
plt.show()

# nonMaxSuprression
print("Filtered Hough votes")
votes = nonMaxSuprression(votes, 20)
plt.imshow(votes)
plt.show()

# Visualize n best matches
n = 10
coords = zip(*np.unravel_index(np.argpartition(votes, -n, axis=None)[-n:], votes.shape))
vis = np.stack(3*[query],2)
print("Detected Positions")
for y,x in coords:
    print(x,y)
    vis = cv2.circle(vis,(x,y), 10, (255,0,0), 2)
utils.show(vis)


# Test your implementation

In [ ]:
import utils
import cv2
import json
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances


def testGHT():
    # Load Images
    query = cv2.imread("data/query.jpg", cv2.IMREAD_GRAYSCALE)
    template = cv2.imread("data/template.jpg", cv2.IMREAD_GRAYSCALE)

    # GHT with search space
    angles = np.linspace(0, 360, 36)
    scales = np.linspace(0.9, 1.3, 10)
    ght = GeneralizedHoughTransform(query, template, angles, scales)

    # Visualize GHT votes
    votes, thetas, s = zip(*ght)
    votes = np.stack(votes).max(0)
    plt.imshow(votes)
    plt.show()

    # Visualize filtered points
    votes = nonMaxSuprression(votes, 20)
    plt.imshow(votes)
    plt.show()

    # Extract n points wiht highest voting score
    n = 10
    coords = list(zip(*np.unravel_index(np.argpartition(votes, -n, axis=None)[-n:], votes.shape)))
    vis = np.stack(3*[query],2)
    for y,x in coords:
        vis = cv2.circle(vis,(x,y), 10, (255,0,0), 2)
    utils.show(vis)

    # Compare with ground-truth centroids
    f = open("centroids.txt", "r")
    centroids = f.read()
    f.close()
    centroids = centroids.split("\n")[:-1]
    centroids = [centroid.split() for centroid in centroids]
    centroids = np.array([[int(centroid[0]),int(centroid[1])] for centroid in centroids])

    # Visualize centroids
    vis = np.stack(3*[query],2)
    for x,y in centroids:
        vis = cv2.circle(vis,(x,y), 10, (255,0,0), 2)
    utils.show(vis)

    # Compute Distances and apply threshold
    coords = np.array(coords)[:,::-1]
    d = euclidean_distances(centroids, coords).min(1)
    correct_detections = np.count_nonzero((d<10))
    score = { "scores": {"Correct_Detections": correct_detections }}

    print(json.dumps(score))

testGHT()